In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from tifffile import imread, imwrite
import numpy as np
import pandas as pd
from skimage.draw import polygon
from pathlib import Path
from loguru import logger

from multiplex_pipeline.utils.config_loader import load_config
from multiplex_pipeline.core_preparation.channel_scanner import discover_channels, build_transfer_map
from multiplex_pipeline.core_preparation.controller import CorePreparationController
from multiplex_pipeline.core_preparation.file_io import GlobusFileStrategy, LocalFileStrategy

c:\Users\KMK280\AppData\Local\miniconda3\envs\napari-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [34]:
df_path = r'R:\Wayne\BLCA\BLCA-1B_Analysis\cores.pkl'

temp_dir = r"R:\Wayne\BLCA\BLCA-1B_Analysis\temp"
output_dir = r"R:\Wayne\BLCA\BLCA-1B_Analysis\temp"

margin = 0
mask_value = 0
max_pyramid_levels = 3

transfer_cleanup_enabled = True
core_cleanup_enabled = True

In [4]:
df = pd.read_pickle(df_path)
df.head()

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,640.0,4992.0,3136.0,8832.0,polygon,"[[2688.0, 6080.0], [4992.0, 3136.0], [640.0, 3..."
1,Core_001,960.0,5632.0,9920.0,15680.0,rectangle,"[[5632.0, 15680.0], [5632.0, 9920.0], [960.0, ..."
2,Core_002,1664.0,5504.0,16896.0,22656.0,rectangle,"[[5504.0, 22656.0], [5504.0, 16896.0], [1664.0..."
3,Core_003,1728.0,5952.0,23680.0,29312.0,rectangle,"[[5952.0, 29312.0], [5952.0, 23680.0], [1728.0..."
4,Core_004,1856.0,6400.0,30400.0,35712.0,rectangle,"[[6400.0, 35712.0], [6400.0, 30400.0], [1856.0..."


In [40]:
df_03 = df.loc[3:3,:]
df_03.reset_index(drop=True, inplace=True)
df_03.to_csv(os.path.join(r'R:\Wayne\BLCA\BLCA-1B_Analysis', 'df_03.csv'))

## Local files

In [5]:
input_dir = r'R:\CellDive\BLCA-1B\BLCA-1B_Final'

In [6]:
channel_map = discover_channels(Path(input_dir), include_channels=[], exclude_channels=[])
#print(channel_map)

2025-06-17 15:40:50.252 | INFO     | multiplex_pipeline.core_preparation.channel_scanner:scan_channels_from_list:45 - Discovered channels before filtering: ['011_CD3', '008_DAPI', '022_CD56', '028_DAPI', '014_CD4', '016_DAPI', '025_CD86', '001_cycD3', '005_CDK6', '017_CD27', '010_p38', '010_p16', '009_DAPI', '012_DAPI', '003_cycD1', '001_pRB', '007_CDT1', '008_pRB', '016_pERK', '013_DAPI', '018_DAPI', '027_CD4', '020_EpCAM', '024_DAPI', '015_CD11b', '014_PD-1', '019_p-p130', '006_Plk1', '020_DAPI', '017_CDC6', '013_PH3', '008_YAP1', '025_DAPI', '022_CD19', '002_cycE2', '017_DAPI', '002_DAPI', '027_FOXP3', '028_p65', '012_IL-8', '004_DAPI', '005_bCat', '026_ERa', '008_CDK2', '001_pH2AX', '008_ECad', '023_CD25', '023_DAPI', '009_p21', '002_cycB2', '011_DAPI', '012_cMYC', '025_ECad', '006_DAPI', '024_PML', '001_CDC25C', '025_p27', '011_CD45', '028_CD86', '011_CCR7', '029_DAPI', '029_CD62L', '026_HER2', '007_wee1', '007_VIM', '027_CD68', '018_TGFb1', '003_cCASP3', '007_CDK4', '024_HLADR', 

In [7]:
selected_keys = ["DAPI", "CD3"]
short_map = {k: channel_map[k] for k in selected_keys if k in channel_map}
short_map

{'DAPI': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [8]:
strategy = LocalFileStrategy()

controller = CorePreparationController(
    metadata_df = df.loc[17:17,:],
    image_paths = short_map,
    temp_dir = temp_dir,
    output_dir = output_dir,
    file_strategy = strategy,
    margin = margin,
    mask_value = mask_value,
    max_pyramid_levels = max_pyramid_levels,
)

controller.run()

2025-06-17 15:40:52.178 | INFO     | multiplex_pipeline.core_preparation.controller:run:46 - Starting controller run loop...
2025-06-17 15:40:56.214 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:79 - Cut and saved core Core_017, channel DAPI
2025-06-17 15:40:57.875 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:79 - Cut and saved core Core_017, channel CD3
2025-06-17 15:40:57.885 | INFO     | multiplex_pipeline.core_preparation.controller:try_assemble_ready_cores:84 - Assembling full core Core_017
2025-06-17 15:41:02.357 | INFO     | multiplex_pipeline.core_preparation.assembler:assemble_core:68 - Core 'Core_017' assembled with channels: ['CD3', 'DAPI']


INFO     The Zarr backing store has been changed from None the new file path:                                      
         R:\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_017.zarr                                                         


2025-06-17 15:43:07.240 | INFO     | multiplex_pipeline.core_preparation.controller:run:65 - All channels processed and cores assembled.


## Files through Globus

In [9]:
from multiplex_pipeline.utils.globus_utils import GlobusConfig, create_globus_tc

In [10]:
input_globus_dir = '/CellDive/BLCA-1B/BLCA-1B_Final'

# make sure that this location is available in your Globus collection
output_dir = r'D:\multiplex_pipeline\tests'
temp_dir = r'D:\multiplex_pipeline\tests'

In [11]:
# get globus config
config_dir = Path(r'D:\\globus_config')
gc = GlobusConfig.from_config_files(config_dir)
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [12]:
channel_map = discover_channels(input_globus_dir, include_channels=[], exclude_channels=[], gc = gc)

2025-06-17 15:43:21.504 | INFO     | multiplex_pipeline.core_preparation.channel_scanner:scan_channels_from_list:45 - Discovered channels before filtering: ['001_pH2AX', '001_pRB', '001_cycD3', '001_DAPI', '001_CDC25C', '010_p38', '010_p16', '010_PDL1', '010_DAPI', '010_CD127', '011_CD3', '011_SKP2', '011_CD45', '011_DAPI', '011_CCR7', '012_CD11C', '012_cMYC', '012_CD20', '012_DAPI', '012_IL-8', '013_CD68', '013_CD69', '013_PH3', '013_DAPI', '013_TNFa', '014_PD-1', '014_CD4', '014_FOXP3', '014_DAPI', '014_Snail1', '015_CD11b', '015_CD8a', '015_NaKATPase', '015_DAPI', '015_Antibody1', '016_CD31', '016_pERK', '016_CD44', '016_DAPI', '017_CD27', '017_pAKT', '017_CDC6', '017_DAPI', '018_GranzymeB', '018_TGFb1', '018_p130', '018_DAPI', '019_CD163', '019_Fibro', '019_p-p130', '019_DAPI', '002_cycA1', '002_cycE1', '002_cycB2', '002_DAPI', '002_cycE2', '020_HLA1', '020_EpCAM', '020_pATR', '020_DAPI', '021_SMA', '021_IL6', '021_53BP1', '021_DAPI', '022_CD19', '022_CD56', '022_DAPI', '023_CD25',

In [13]:
selected_keys = ["DAPI", "CD3"]
short_map = {k: channel_map[k] for k in selected_keys if k in channel_map}
short_map

{'DAPI': '/CellDive/BLCA-1B/BLCA-1B_Final/BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': '/CellDive/BLCA-1B/BLCA-1B_Final/BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [16]:
# build transfer map
transfer_cache_dir = Path(temp_dir) / "input_cache"
transfer_map = build_transfer_map(short_map, transfer_cache_dir)

# build a dict for transfered images
image_paths = {
    ch: str(Path(transfer_cache_dir) / Path(remote).name)
    for ch, (remote, _) in transfer_map.items()
}
image_paths

{'DAPI': 'D:\\multiplex_pipeline\\tests\\input_cache\\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': 'D:\\multiplex_pipeline\\tests\\input_cache\\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [36]:
strategy = GlobusFileStrategy(tc=tc, transfer_map=transfer_map, gc=gc, cleanup_enabled = transfer_cleanup_enabled)

controller = CorePreparationController(
    metadata_df = df.loc[15:15,:],
    image_paths = image_paths,
    temp_dir = temp_dir,
    output_dir = output_dir,
    file_strategy = strategy,
    margin = margin,
    mask_value = mask_value,
    max_pyramid_levels = max_pyramid_levels,
    core_cleanup_enabled = core_cleanup_enabled,
)

controller.run()

2025-06-17 16:32:35.669 | INFO     | multiplex_pipeline.core_preparation.file_io:_submit_all_transfers:65 - Submitted transfer for DAPI to /~/D/multiplex_pipeline/tests/input_cache/BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif (task_id=32598a4f-4bba-11f0-a701-0affcfc1d1e5)
2025-06-17 16:32:36.056 | INFO     | multiplex_pipeline.core_preparation.file_io:_submit_all_transfers:65 - Submitted transfer for CD3 to /~/D/multiplex_pipeline/tests/input_cache/BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif (task_id=32b53b46-4bba-11f0-9834-0affcfc1d1e5)
2025-06-17 16:32:36.097 | INFO     | multiplex_pipeline.core_preparation.controller:run:48 - Starting controller run loop...
2025-06-17 16:33:16.366 | INFO     | multiplex_pipeline.core_preparation.file_io:fetch_or_wait:100 - Transfer for CD3 complete: D:\multiplex_pipeline\tests\input_cache\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif
2025-06-17 16:33:17.608 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:81 - Cu

INFO     The Zarr backing store has been changed from None the new file path:                                      
         R:\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015.zarr                                                         


2025-06-17 16:34:18.002 | DEBUG    | multiplex_pipeline.core_preparation.assembler:_cleanup_core_files:89 - Deleted intermediate TIFF: R:\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015\CD3.tiff
2025-06-17 16:34:18.068 | DEBUG    | multiplex_pipeline.core_preparation.assembler:_cleanup_core_files:89 - Deleted intermediate TIFF: R:\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015\DAPI.tiff
2025-06-17 16:34:18.068 | INFO     | multiplex_pipeline.core_preparation.controller:run:67 - All channels processed and cores assembled.
